In [230]:
# pip install torch

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.2/890.2 MB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 KB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 92.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.0 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from sklearn.metrics import f1_score, fbeta_score, precision_score, recall_score, accuracy_score
# from build_model import initialize_model, load_split_data, build_loader, evaluate, train_model
# from build_model import plot_confusion
import os
from sklearn.metrics import confusion_matrix


In [2]:
MODEL_DIR = '/home/ec2-user/SageMaker/teledermatologyAI_capstone/model/gridsearch'


In [3]:
cd $MODEL_DIR

/home/ec2-user/SageMaker/teledermatologyAI_capstone/model/gridsearch


In [4]:
# Load files in directory, filter to pred files
files = pd.Series(os.listdir('./'), name = "file")
files = pd.DataFrame(files[files.str.contains('pkl')])

# extract model number and name, convert model num to int
files['model_number'] = files.file.apply(lambda st: st[st.find("GS")+2:st.find("_pred")])
files['tuned_model'] = files.file.apply(lambda st: st[:st.find("_pred")])
files['model_number'] = files['model_number'].astype('int')
files.set_index('model_number', inplace = True)

# read in prior partial results
gs = pd.read_csv('gridsearch1.csv')

In [5]:
# Create results df by combining hyperparams and model name
results = pd.concat([files, gs], axis = 1).set_index('model')

In [9]:
results = results[:1]

In [10]:
# new columns
results['pretrained_model'] = 'resnet'
results['optimizer'] = 'SGD'
results['epochs'] = 3
results['num_classes'] = 5
results['batch_size'] = 64
results['workers'] = 24
results['train_time'] = 'unk'
results['data_split'] = 'split_3'
results['label_set'] = 'label_0'
results['accur'] = None
results['F1'] = None
results['F0.5'] = None
results['F2'] = None
results['benign_accur'] = None
results['noncancerous_accur'] = None
results['malignant_accur'] = None
results['infection_accur'] = None
results['unclassified_accur'] = None



/tmp/ipykernel_64876/2023296353.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['pretrained_model'] = 'resnet'
/tmp/ipykernel_64876/2023296353.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['optimizer'] = 'SGD'
/tmp/ipykernel_64876/2023296353.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [11]:
# i = 1
for index, row in results.iterrows():
#     row.iloc[index]['accur'] = i
    
    d = pd.read_pickle(row['file'])
    
    # Scores
    acc = accuracy_score(d.lab, d.pred)
    f1 = f1_score(d.lab, d.pred, average = 'macro')
    f2 = fbeta_score(d.lab, d.pred, average = 'macro', beta = 2)
    f5 =  fbeta_score(d.lab, d.pred, average = 'macro', beta = .5)
    prec = precision_score(d.lab, d.pred, average = 'macro')
    rec = recall_score(d.lab, d.pred, average = 'macro')
    
    # confusion matrix 
    c_matrix = confusion_matrix(d.lab, d.pred, normalize = 'true')

    results.at[index, 'accur'] = acc
    results.at[index, 'F1'] = f1
    results.at[index, 'F0.5'] = f5
    results.at[index, 'F2'] = f2
    results.at[index, 'benign_accur'] = c_matrix[0,0]
    results.at[index, 'noncancerous_accur'] = c_matrix[1,1]
    results.at[index, 'malignant_accur'] = c_matrix[2,2]
    results.at[index, 'infection_accur'] = c_matrix[3,3]
    results.at[index, 'unclassified_accur'] = c_matrix[4,4]
#     i+=1

In [12]:
results.to_csv('./results_gs2.csv')

In [33]:
a = model_scores(d.lab, d.pred)

In [28]:
def model_scores(true_labs, preds):
    acc = accuracy_score(d.lab, d.pred)
    f1 = f1_score(d.lab, d.pred, average = 'macro')
    f2 = fbeta_score(d.lab, d.pred, average = 'macro', beta = 2)
    f5 =  fbeta_score(d.lab, d.pred, average = 'macro', beta = .5)
    prec = precision_score(d.lab, d.pred, average = 'macro')
    rec = recall_score(d.lab, d.pred, average = 'macro')
    
    # confusion matrix & Diags
    c_matrix = confusion_matrix(d.lab, d.pred, normalize = 'true')
    d_0 = c_matrix[0,0]
    d_1 = c_matrix[1,1]
    d_2 = c_matrix[2,2]
    d_3 = c_matrix[3,3]
    d_4 = c_matrix[4,4]
    
    return acc, f1, f2, f5, prec, rec, d_0, d_1, d_2, d_3, d_4

In [35]:
a[0]

0.7022222222222222